# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,magistral'nyy,56.1725,107.4458,-33.13,94,70,0.56,RU,1702264827
1,1,adamstown,-25.0660,-130.1015,21.31,56,0,2.58,PN,1702264828
2,2,dunedin,-45.8742,170.5036,16.40,59,35,3.52,NZ,1702264828
3,3,mcpherson,38.3708,-97.6642,3.38,71,11,6.30,US,1702264717
4,4,sassandra,4.9500,-6.0833,25.63,86,2,1.73,CI,1702264828


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    hover_cols=['City', 'Humidity'],
    size='Humidity',
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=800,
    frame_height=500,
    tiles=True
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &  # Max temperature between 21 and 27 degrees Celsius
    (city_data_df['Wind Speed'] < 4.5) &  # Wind speed less than 4.5 m/s
    (city_data_df['Cloudiness'] == 0)  # Zero cloudiness
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,adamstown,-25.0660,-130.1015,21.31,56,0,2.58,PN,1702264828
53,53,san sebastian de la gomera,28.0916,-17.1133,25.20,23,0,1.54,ES,1702264837
109,109,cabo san lucas,22.8909,-109.9124,24.86,65,0,3.09,MX,1702264738
185,185,margaret river,-33.9500,115.0667,25.26,31,0,2.43,AU,1702264852
246,246,suncho corral,-27.9336,-63.4294,21.37,57,0,4.48,AR,1702264857


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,adamstown,-25.0660,-130.1015,21.31,56,0,2.58,PN,1702264828,
53,53,san sebastian de la gomera,28.0916,-17.1133,25.20,23,0,1.54,ES,1702264837,
109,109,cabo san lucas,22.8909,-109.9124,24.86,65,0,3.09,MX,1702264738,
185,185,margaret river,-33.9500,115.0667,25.26,31,0,2.43,AU,1702264852,
246,246,suncho corral,-27.9336,-63.4294,21.37,57,0,4.48,AR,1702264857,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'radius': radius,
    'limit': 20,  # Limit the results to 1 (the closest hotel)
    "categories":"accommodation.hotel",
    'apiKey': "7338a7c23b174decb04576a162f89bee",
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
cabo san lucas - nearest hotel: Comfort Rooms
margaret river - nearest hotel: Margaret River Hotel
suncho corral - nearest hotel: No hotel found
igaci - nearest hotel: Pousada Todo Dia
al wajh - nearest hotel: فندق رالين 2
piacabucu - nearest hotel: O Leão


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,adamstown,-25.0660,-130.1015,21.31,56,0,2.58,PN,1702264828,No hotel found
53,53,san sebastian de la gomera,28.0916,-17.1133,25.20,23,0,1.54,ES,1702264837,Hotel Villa Gomera
109,109,cabo san lucas,22.8909,-109.9124,24.86,65,0,3.09,MX,1702264738,Comfort Rooms
185,185,margaret river,-33.9500,115.0667,25.26,31,0,2.43,AU,1702264852,Margaret River Hotel
246,246,suncho corral,-27.9336,-63.4294,21.37,57,0,4.48,AR,1702264857,No hotel found
252,252,igaci,-9.5369,-36.6336,22.23,81,0,2.64,BR,1702264857,Pousada Todo Dia
422,422,al wajh,26.2455,36.4525,21.97,60,0,2.57,SA,1702264880,فندق رالين 2
482,482,piacabucu,-10.4056,-36.4344,25.03,77,0,3.42,BR,1702264719,O Leão


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    size='Humidity',
    title='City Humidity with Hotel Information',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=800,
    frame_height=500,
    tiles=True
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)